In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np

parks = pd.read_csv("../Resources/state-park-facility-points-1.csv")
attendance = pd.read_csv("../Resources/State_Park_Annual_Attendance_Figures_modified.csv")
income = pd.read_csv("../Resources/Median Household Income by County.csv")
population = pd.read_csv("../Resources/Population Estimates by County.csv")

In [4]:
parks = parks.drop(columns=['Facility URL', 'Golf URL', 'Nature Center URL', 'Location']).fillna('N')

In [5]:
parks = parks.sort_values(by='Name')
parks
parks.to_csv(r'processed/StatePark.csv', index = False, header=True)

In [6]:
att2019 = attendance[attendance["Year"] == 2019]
att2019.rename(columns={'Facility': 'Name'}, inplace=True)
att2019 = att2019.sort_values(by = 'Name')
grouped_att = att2019.groupby(['County'])['Attendance'].sum()
grouped_county = att2019.groupby(['County'])['County']

park_att = pd.DataFrame({'County': grouped_county , 'Attendance': grouped_att})

# park_att['County']= park_att.index
# park_att.reset_index(drop=True, inplace=True)
# df['text_new'] = df['text'].str.split('::').str[0]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  v = np.array(v, copy=False)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1672: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'd

In [7]:
income['Median Household Income by County'] = income['Median Household Income by County'].str.replace('County', '')
income.drop(income.tail(1).index,inplace=True)
income.drop(income.head(1).index,inplace=True)
income.rename(columns={'Median Household Income by County':'County', 'Unnamed: 1': 'Income', 'Unnamed: 2': 'Margin of Error'}, inplace=True)
income.to_csv (r'processed/income.csv', index = False, header=True)

In [8]:
population['Population Estimates by County'] = population['Population Estimates by County'].str.replace('County', '')
population.drop(income.tail(1).index,inplace=True)
population.drop(income.head(1).index,inplace=True)
population.rename(columns={'Population Estimates by County':'County', 'Unnamed: 1': 'Population'}, inplace=True)

In [9]:
population.to_csv (r'processed/population.csv', index = False, header=True)

In [10]:
att2019.to_csv (r'processed/2019_Attendance.csv', index = False, header=True)
park_att.to_csv (r'processed/park_county_attendance.csv', index = False, header=True)


In [11]:
import sqlite3 as sql
cleaned_data = pd.read_csv("processed/county_attendance_income_pop.csv")
conn = sql.connect('newyork.db')

In [12]:
conn.execute("DROP TABLE IF EXISTS county_data")

conn.execute('''CREATE TABLE county_data (
    County TEXT PRIMARY KEY,  
    Attendance INT, 
    Income INT, 
    Population INT
)''')

In [13]:
# cleaned_data['Income'] = cleaned_data['Income'].str.replace('$', '')
# cleaned_data['Income'] = cleaned_data['Income'].str.replace(',', '')
# cleaned_data['Population'] = cleaned_data['Population'].str.replace(',', '')
# cleaned_data = cleaned_data.drop(columns=['Margin of Error'])

cleaned_data
cleaned_data.to_sql('county_data', conn, if_exists='append', index= False)
cleaned_data.to_csv('county_data', index= False)
explore = pd.read_sql("SELECT * FROM county_data", conn)
explore

,County,Attendance,Income,Population
0,Albany,692080,66252,305506
1,Allegany,50133,48412,46091
2,Bronx,339868,40088,1418207
3,Broome,252508,52226,190488
4,Cattaraugus,1521210,48703,76117
...,...,...,...,...
57,Washington,8588,57258,61204
58,Wayne,80599,59449,89918
59,Westchester,2351019,96610,967506
60,Wyoming,27031,58052,39859


In [17]:
yearly_attendance = pd.read_csv("../Resources/State_Park_Annual_Attendance_Figures_modified.csv")
yearly_attendance = yearly_attendance.sort_values(by='Year').drop(columns=['OPRHP Region', 'County'])

yearly_attendance.to_csv (r'processed/yearly_attendance.csv', index = False, header=True)


In [18]:
conn.execute("DROP TABLE IF EXISTS attendance_data")

conn.execute('''CREATE TABLE attendance_data (
    No INT PRIMARY KEY,  
    Year INT, 
    County TEXT, 
    Facility INT,
    Attendance INT
)''')

In [25]:
all_attendance = pd.read_csv("../Resources/State_Park_Annual_Attendance_Figures_modified.csv")
all_attendance["No"] = np.arange(0,len(all_attendance))
all_attendance = all_attendance.drop(columns = ["OPRHP Region"])
all_attendance = all_attendance.loc[~all_attendance['Facility'].str.contains("Robert Moses St Pk")]
all_attendance

,Year,County,Facility,Attendance,No
0,2020,Tompkins,Allan Treman Marina,341963,0
1,2019,Tompkins,Allan Treman Marina,355313,1
2,2018,Tompkins,Allan Treman Marina,291507,2
3,2017,Tompkins,Allan Treman Marina,311260,3
4,2016,Tompkins,Allan Treman Marina,318309,4
...,...,...,...,...,...
4246,2007,Erie,Woodlawn Beach,180326,4246
4247,2006,Erie,Woodlawn Beach,142245,4247
4248,2005,Erie,Woodlawn Beach,169049,4248
4249,2004,Erie,Woodlawn Beach,151359,4249


In [26]:

all_attendance = all_attendance[['No', 'Year', 'County', 'Facility', 'Attendance']]
all_attendance.head()
all_attendance.to_sql('attendance_data', conn, if_exists='append', index= False)

In [27]:
explore2 = pd.read_sql("SELECT * FROM attendance_data", conn)
explore2

,No,Year,County,Facility,Attendance
0,0,2020,Tompkins,Allan Treman Marina,341963
1,1,2019,Tompkins,Allan Treman Marina,355313
2,2,2018,Tompkins,Allan Treman Marina,291507
3,3,2017,Tompkins,Allan Treman Marina,311260
4,4,2016,Tompkins,Allan Treman Marina,318309
...,...,...,...,...,...
4210,4246,2007,Erie,Woodlawn Beach,180326
4211,4247,2006,Erie,Woodlawn Beach,142245
4212,4248,2005,Erie,Woodlawn Beach,169049
4213,4249,2004,Erie,Woodlawn Beach,151359
